# 1. 데이터 로드
## 1.1 데이터 로드 및 확인

In [1]:
import pandas as pd
from util import Utils as ut

train = pd.read_csv('input/funda_train.csv')

train.head()

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000


In [2]:
# 데이터 정보 확인
train.info() # 인트형 변수가 3개 존재한다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6556613 entries, 0 to 6556612
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   store_id          int64  
 1   card_id           int64  
 2   card_company      object 
 3   transacted_date   object 
 4   transacted_time   object 
 5   installment_term  int64  
 6   region            object 
 7   type_of_business  object 
 8   amount            float64
dtypes: float64(1), int64(3), object(5)
memory usage: 450.2+ MB


In [3]:
# 결착 데이터 확인
train.isnull().sum()

store_id                  0
card_id                   0
card_company              0
transacted_date           0
transacted_time           0
installment_term          0
region              2042766
type_of_business    3952609
amount                    0
dtype: int64

In [4]:
train.isnull().mean() # 결착율이 31% 60% 인 컬럼이 존재한다. (데이터 튜닝이 필요하다)

store_id            0.000000
card_id             0.000000
card_company        0.000000
transacted_date     0.000000
transacted_time     0.000000
installment_term    0.000000
region              0.311558
type_of_business    0.602843
amount              0.000000
dtype: float64

<b>[결착율 높은 데이터 분석]</b><br>
1. 결착율이 높은 데이터는 지역('region') 과 업소형태('type_of_business')이다.
2. 'store_id' 가 같은데 'region' 혹은 'type_of_business' 가 NaN 일 경우 다른 컬럼값을 이용해 채워줄 수 있으므로 조사해본다.

In [5]:
# column_key 로 주어진 컬럼은 'store_id' 와 일치할 경우 동일한 값을 유지해야 한다.
# 일부만 누락되어 있는지 확인하기 위해 조사
def CheckEmptyItem( df, column_key ) :
    
    store_id_cnt = 0
    empty_item_cnt = 0
    for store_id in range(train['store_id'].max()+1) :
        # store_id 개수와 empty_item 개수를 파악해 일치하지 않을 경우를 찾는다.
        store_id_cnt = len( train[train['store_id']==store_id].index )
        empty_item_cnt = train[train['store_id'] == store_id][column_key].isnull().sum()
        
        # store_id 를 보유한 행이 존재하고, empty_item 개수가 0이 아니며 store_id 와 empty_item 개수가 다르다면 일부만 누락된 행이다.
        if 0 != store_id_cnt and 0 != empty_item_cnt and store_id_cnt != empty_item_cnt :
            print( store_id )

In [6]:
# type_of_business는 store_id 에 매칭되지만 유실된 NaN 있을 경우 같은 값을 채워주기 위해 조사
CheckEmptyItem( train, 'type_of_business' )

In [7]:
# type_of_business는 store_id 에 매칭되지만 유실된 NaN 있을 경우 같은 값을 채워주기 위해 조사
CheckEmptyItem( train, 'region' )

<b>[조사 결과]</b><br>
1. 결국 결착된 데이터는 전부 누락된 데이터이기 때문에 임의의 데이터로 채워주거나 drop 을 고려한다.
2. 임의의 데이터로 채운 후 조사해보고, 드압한 후 테스트해 결과를 보고 결정한다.

# 2. 데이터 전처리
## 2.1 정수형 변수 조사 및 리타입

In [8]:
# 미리 제작해둔 유틸을 이용해 DataFrame Column 별 정수형변수의 최대 최소값을 출력한다.
ut.convert_data_type( train, mode='print' )

key: store_id
	min: 0
	max: 2136
key: card_id
	min: 0
	max: 4663856
key: installment_term
	min: 0
	max: 93


,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000
...,...,...,...,...,...,...,...,...,...
6556608,2136,4663855,d,2019-02-28,23:20,0,제주 제주시,기타 주점업,-4500.000000
6556609,2136,4663855,d,2019-02-28,23:24,0,제주 제주시,기타 주점업,4142.857143
6556610,2136,4663489,a,2019-02-28,23:24,0,제주 제주시,기타 주점업,4500.000000
6556611,2136,4663856,d,2019-02-28,23:27,0,제주 제주시,기타 주점업,571.428571


<b>[조사 결과]</b><br>
1. 최대 최소값이 전부 int32 보다 작은 값에 들어가기 때문에 데이터 타입을 변경한다.

In [11]:
train = ut.convert_data_type( train, mode='convert' )
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6556613 entries, 0 to 6556612
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   store_id          int16  
 1   card_id           int32  
 2   card_company      object 
 3   transacted_date   object 
 4   transacted_time   object 
 5   installment_term  int8   
 6   region            object 
 7   type_of_business  object 
 8   amount            float64
dtypes: float64(1), int16(1), int32(1), int8(1), object(5)
memory usage: 343.9+ MB


<b>변경 후 메모리 사용량이 450MB 에서 344MB 로 줄어든 것을 확인할 수 있다.</b>

## 2.2 최근 폐점한 매장이 있는지 조사

In [26]:
# 최근 한달간 매출이 0이라면 폐점한 매장이다.
# 매출 예측값을 0으로 지정할 수 있도록 처리한다.
df_temp = (train['transacted_date'] > '2019-01-31') & (train['amount'] > 0)
len(train[df_temp]['store_id'].unique())

1925

In [28]:
len(train['store_id'].unique()) - len(train[df_temp]['store_id'].unique())

42